|<font size="5">Classification</font>|
|------------------------------------|

### Import

In [1]:
from config import *
from src.data import subjects
import numpy as np
import pandas as pd
import glob
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

### Read Data

In [2]:
all_sub_list = subjects.to_group_level_analysis[0]
all_subjects = subjects.subjects['all'][subjects.subjects['all']['participant_id'].isin(all_sub_list)]

subjects_list = {}
for sg in subjects_groups:
    subjects_list[sg] = all_subjects.where(all_subjects.group==sg).dropna()
    print('\n%d %s subjects are included'%(len(subjects_list[sg]),sg))
    display(subjects_list[sg])
    subjects_list[sg] = subjects_list[sg].participant_id.tolist()


24 CN subjects are included


,group,participant_id,sex,age
0,CN,003S4288,F,78.4
1,CN,011S0021,F,84.9
2,CN,020S6185,M,82.7
3,CN,023S1190,F,87.4
4,CN,035S6160,M,60.7
5,CN,036S6189,F,69.6
6,CN,037S4028,F,70.7
7,CN,041S4200,F,76.4
8,CN,041S6159,M,76.3
9,CN,041S6192,F,83.3



0 EMCI subjects are included


,group,participant_id,sex,age



0 LMCI subjects are included


,group,participant_id,sex,age



24 AD subjects are included


,group,participant_id,sex,age
72,AD,002S5018,M,73.4
73,AD,006S4153,M,81.5
74,AD,006S4867,M,75.7
75,AD,013S5071,M,76.4
76,AD,018S4696,F,73.3
77,AD,018S4733,M,75.4
78,AD,018S5074,F,74.9
79,AD,019S4252,F,86.8
80,AD,019S4477,F,82.4
81,AD,019S4549,M,79.4


In [3]:
X = pd.read_csv(rootdir + '/data/07_features/fsisher_fsf/fsisher_fsf_features.csv')
y  = pd.read_csv(rootdir + '/data/07_features/fsisher_fsf/targets.csv')
X = X.set_index('participant_id')
y = y.set_index('participant_id')

In [4]:
y

,group
participant_id,
002S4225,0
002S4262,0
002S5018,1
003S4288,0
006S4153,1
006S4867,1
011S0021,0
013S5071,1
018S4313,0


### Split Data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.3, train_size=.7)

In [6]:
y_test  = y_test.group.to_numpy()
y_train = y_train.group.to_numpy()
X_test  = X_test.to_numpy()
X_train = X_train.to_numpy()

In [7]:
#X_test

#### SVM

In [8]:
krnls = ['linear', 'poly', 'rbf']

svm_dic = {
    'linear': 
    {'train':[], 'test':[]},
    'poly':
     {'train':[], 'test':[]},
    'rbf': 
    {'train':[], 'test':[]}
}


for k in krnls:
    clf = svm.SVC(kernel=k)
    clf.fit(X_train, y_train)
    
    test_prediction = clf.predict(X_test)
    train_prediction = clf.predict(X_train)
 
    svm_dic[k]['test'].append(metrics.accuracy_score(
        test_prediction, y_test))
    svm_dic[k]['test'].append(metrics.precision_score(
        test_prediction, y_test))
    svm_dic[k]['test'].append(metrics.recall_score(
        test_prediction, y_test, zero_division = 1))
    
    svm_dic[k]['train'].append(metrics.accuracy_score(
        train_prediction, y_train))
    svm_dic[k]['train'].append(metrics.precision_score(
        train_prediction, y_train))
    svm_dic[k]['train'].append(metrics.recall_score(
        train_prediction, y_train, zero_division = 1))

In [9]:
svm_dic

{'linear': {'train': [0.9090909090909091, 0.8421052631578947, 1.0],
  'test': [0.8, 0.8, 0.6666666666666666]},
 'poly': {'train': [1.0, 1.0, 1.0], 'test': [0.8, 0.8, 0.6666666666666666]},
 'rbf': {'train': [0.9393939393939394, 0.8947368421052632, 1.0],
  'test': [0.8666666666666667, 1.0, 0.7142857142857143]}}